In [3]:
#This code contains the functions required to work with Laniakea 

#the following ones are used to compute the radial velocity and its error for any point in Laniakea 
#given its coordinates in sgx, sgy, sgz

def velocitycell(_ii,_jj,_kk):
    
    lania_vv_cell = t[:, _kk, _jj, _ii]
    
    return lania_vv_cell

def errvelocitycell(_ii,_jj,_kk):
    lania_vv_cell_err = t_e[_kk, _jj, _ii]
    return lania_vv_cell_err

# Requires coordinates for the point p to be given as numbers (not arrays)

def lania_v_point(_xp, _yp, _zp, _addbf=True):

    if _addbf:
        bulkflow = (sgx_bf,sgy_bf,sgz_bf)    
    else:
        bulkflow = (0,0,0)
       
    X=32. + (64./1000.)*(_xp)
    Y=32. + (64./1000.)*(_yp)
    Z=32. + (64./1000.)*(_zp)
    
    IX = int(X)
    IY = int(Y)
    IZ = int(Z)

    DX = X-IX
    DY = Y-IY
    DZ = Z-IZ
    TX = 1-DX
    TY = 1-DY
    TZ = 1-DZ

     
    v_point_cic =  (velocitycell(IX,IY,IZ) -bulkflow)*TX*TY*TZ + (velocitycell(IX+1,IY,IZ+1) -bulkflow)*TX*TY*DZ + (velocitycell(IX,IY+1,IZ) -bulkflow)*TX*DY*TZ 
    + (velocitycell(IX,IY+1,IZ+1) -bulkflow)*TX*DY*DZ + (velocitycell(IX+1,IY,IZ) -bulkflow)*DX*TY*TZ + (velocitycell(IX+1,IY,IZ+1)*DX*TY*DZ -bulkflow) 
    + (velocitycell(IX+1,IY+1,IZ) -bulkflow)*DX*DY*TZ + (velocitycell(IX+1,IY+1,IZ+1) -bulkflow)*DX*DY*DZ
    
    return v_point_cic



#The following can compute the error of each cell using CIC. However, in the end we decided to use 
#a flat error of 170 km/s for every point

def lania_v_point_err(_xp, _yp, _zp):
    
    nX=32. + (64./1000.)*(_xp)
    nY=32. + (64./1000.)*(_yp)
    nZ=32. + (64./1000.)*(_zp)
    
    nIX = int(nX)#.astype(int)
    nIY = int(nY)#.astype(int)
    nIZ = int(nZ)#.astype(int)

    nDX = nX-nIX
    nDY = nY-nIY
    nDZ = nZ-nIZ
    nTX = 1-nDX
    nTY = 1-nDY
    nTZ = 1-nDZ

  
    v_err_cic = np.sqrt( ((nTX*nTY*nTZ)**2)*(errvelocitycell(nIX,nIY,nIZ)**2) +((nTX*nTY*nDZ)**2)*(errvelocitycell(nIX+1,nIY,nIZ+1)**2) 
    + ((nTX*nDY*nTZ)**2)*(errvelocitycell(nIX,nIY+1,nIZ)**2)   +((nTX*nDY*nDZ)**2)*(errvelocitycell(nIX,nIY+1,nIZ+1)**2) 
    +((nDX*nTY*nTZ)**2)*(errvelocitycell(nIX+1,nIY,nIZ)**2)  +((nDX*nTY*nDZ)**2)*(errvelocitycell(nIX+1,nIY,nIZ+1)**2)  
    +((nDX*nDY*nTZ)**2)*(errvelocitycell(nIX+1,nIY+1,nIZ)**2)+((nDX*nDY*nDZ)**2)*(errvelocitycell(nIX+1,nIY+1,nIZ+1)**2))
    
#     v_err_cic=np.full(shape=len(t_e),fill_value=170)

    return v_err_cic     
    
# The following computes the radial velocity of the point p with respect to the point c

def lania_radial_velocity(_xp,_yp,_zp,_xc,_yc,_zc,_addbf=True,_addc=True):
 
    if _addc:
        epsilon=1
    else:
        epsilon=0
    if _addbf:
        BF=True
    else:
        BF=False
        
    radialvelocity= ( (lania_v_point(_xp,_yp,_zp,BF)[2]-epsilon*lania_v_point(_xc,_yc,_zc,BF)[2])*(_xp-_xc) 
                     +(lania_v_point(_xp,_yp,_zp,BF)[1]-epsilon*lania_v_point(_xc,_yc,_zc,BF)[1])*(_yp-_yc)
                     +(lania_v_point(_xp,_yp,_zp,BF)[0]-epsilon*lania_v_point(_xc,_yc,_zc,BF)[0])*(_zp-_zc))/(
                     np.sqrt( (_xp-_xc)**2+(_yp-_yc)**2+(_zp-_zc)**2))
    
    return radialvelocity

def lania_rad_vel_err(_xp,_yp,_zp,_xc,_yc,_zc,_addc=True):
    if _addc:
        epsilon=1
    else:
        epsilon=0
            
    sumerr= np.sqrt((lania_v_point_err(_xp, _yp, _zp))**2.0+ epsilon*(lania_v_point_err(_xc, _yc, _zc))**2.0 + 170**2)
    
    return sumerr    

def distancefromcenter(_xp,_yp,_zp,_xc,_yc,_zc):
    
    distance= np.sqrt((_xp-_xc)**2 +(_yp -_yc)**2 + (_zp-_zc)**2 )
    return distance   




In [ ]:
#